# Introduction to AVISO-data

The following notebook highlights how to import the [AVISO data set](http://aviso.oceanobs.com) (2D satellite-altimetry-derived ocean surface current product). The data is freely available from the [Copernicus Marine Environment](https://marine.copernicus.eu).

The sea surface height $ h $ acts as a stream function for the geostrophic ocean surface velocity field. Particle trajectories on the ocean surface are approximately trajectories of the 2D system of ODEs:

\begin{align}
\dot{\varphi} &= -\dfrac{g}{R^2f(\theta)cos(\theta)}\partial_{\theta}h(\phi, \theta, t) \\
\dot{\theta} &= \dfrac{g}{R^2f(\theta)cos(\theta)}\partial_{\varphi}h(\phi, \theta, t),
\end{align}

where $ \varphi, \theta $ respectively are the longitudinal and latitudinal position of the particle, $ g $ is the constant of gravity, $ R $ is the mean radius, $ f(\theta) = 2\Omega \sin(\theta) $ is the coriolis parameter and $ \Omega $ is the mean angular velocity of the earth.

The AVISO-data provides global coverage of ocean currents at daily time-intervals with a spatial resolution of $ 0.25^{\circ} $. As part of this work, we focus the commonly studied Agulhas region in the Southern Ocean:

\begin{equation}
(\varphi, \theta) \in [14^{\circ}W, 9^{\circ}E] \times [39^{\circ}S, 21^{\circ}S]
\end{equation}

over a period of (maximum) 90 days starting at $ t_0 = 24 $ Novembre 2006.

# Import AVISO Data from ".nc" file

After downloading the AVISO dataset in the region and time interval of interest, the ".nc" file is saved in the following [folder](http://localhost:8888/notebooks/TBarrier/2D/Gridded/codes_data/data/files). For a tutorial about how to import ".nc"-files in python we refer to the following [link](#https://unidata.github.io/netcdf4-python/). The function **Import_AVISO** briefly highlights the steps involved when importing the AVISO dataset.

In [1]:
from netCDF4 import Dataset, num2date
from datetime import datetime
import numpy as np

def Import_AVISO(file):

    # Suppress DeprecationWarning
    import warnings
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", category=DeprecationWarning)
    
        fileobj = Dataset(file, 'r')

        # x: longitudinal coordinate (in degrees)
        x = fileobj.variables['longitude'][:]
        
        # y: latitudinal coordinate (in degrees)
        y = fileobj.variables['latitude'][:]
        
        # u: zonal velocity component (in m/s)
        u = fileobj.variables['ugos'][:,:,:].transpose((1, 2, 0))
        
        # v: meridional velocity component (in m/s)
        v = fileobj.variables['vgos'][:,:,:].transpose((1, 2, 0))
        
        # time
        nctime = fileobj.variables['time'][:]
        t_unit = fileobj.variables['time'].units  # get unit  "days since 1950-01-01T00:00:00Z"

        t_cal = fileobj.variables['time'].calendar

        date_gregorian = [num2date(nctime[i], units=t_unit, calendar=t_cal) for i in range(len(nctime))]
        # time (in days)
        time = (np.array(nctime)) - nctime[0]
            
    return x, y, u, v, time

In [2]:
# Import data from '.nc' file

x, y, u, v, t = Import_AVISO('../Data/AVISO_Agulhas.nc')

# Convert velocity units from $ \dfrac{meters}{second} $ to $ \dfrac{degrees}{day} $

As the ocean velocity is given in units of $ \dfrac{meters}{second} $, but the spatial coordinates are given in $ \dfrac{degrees}{day} $, we need to account for this missmatch in the units.

In [3]:
# Radius of earth
earthRadius = 6371*(10**3)

# Define velocities (in degrees/day)
U, V = np.zeros(u.shape), np.zeros(v.shape)
    
# Iterate over all velocities in y-direction
for i in range(U.shape[0]):
    
    # Iterate over all velocities in x-direction
    for j in range(V.shape[1]):
    
        # iterate over all velocities in time
        for k in range(V.shape[2]):
            
            # compute longitudinal component of velocity (in degrees/day)
            U[i, j, k] = (u[i, j, k] / (np.cos(y[i]*(np.pi/180))*(earthRadius)))*180*3600*24/np.pi
            # compute latitudinal component of velocity (in degrees/day)
            V[i, j, k] = (v[i, j, k] / earthRadius)*180*3600*24/np.pi

# Save data in a '.mat' file

In [4]:
import scipy.io

# save data to 'Agulhas_AVISO.mat' file in folder 'Data'
scipy.io.savemat('Agulhas_AVISO.mat', {'u': U, 'v': V, 'x': x, 'y': y, 't': t})